In [4]:
import os
import io

import rioxarray
import xarray as xr
import numpy as np
from pyproj import Transformer
from shapely.geometry import Point, shape
from shapely import ops
import matplotlib.pyplot as plt
from PIL import Image as PILImage
import bqplot

import ipywidgets as ipw
from IPython.display import display, clear_output

In [5]:
cube = xr.open_dataset('/home/loic/Downloads/czechia_nrt_test.nc')

In [10]:
fc = [{'type': 'Feature',
       'geometry': {'type': 'Point', 'coordinates': [4813210, 2935950]},
       'properties': {'idx': 1}},
      {'type': 'Feature',
       'geometry': {'type': 'Point', 'coordinates': [4815170, 2936650]},
       'properties': {'idx': 2}}]

class ColorComposite(object):
    """Transform to create a stretched image in numpy format from a multivariate xarray Dataset

    Works only for a Dataset with a single temporal slice (e.g. only x and y coordinate valiables)
    """
    def __init__(self, b='B02_20', g='B03_20', r='B04_20',
                 blim=[20,2000], glim=[50,2000], rlim=[20,2000]):
        self.b = b
        self.g = g
        self.r = r
        self.blim = blim
        self.glim = glim
        self.rlim = rlim

    @staticmethod
    def stretch(arr, blim=[20,2000], glim=[50,2000], rlim=[20,2000]):
        """Apply color stretching and [0,1] clipping to a 3 bands image
    
        Args:
            arr (np.ndarray): 3D array; bands as last dimension in RGB order.
                (if read in 'wavelength order' satellite images are usually BGR)
            blim,glim,rlim (list): min and max values between which to stretch the individual bands
        """
        bottom = np.array([[[rlim[0], glim[0], blim[0]]]])
        top = np.array([[[rlim[1], glim[1], blim[1]]]])
        arr_stretched = (arr - bottom)/(top-bottom)
        return np.clip(arr_stretched, 0.0, 1.0)

    def __call__(self, ds):
        bgr = np.stack([ds[self.b].values, ds[self.g].values, ds[self.r].values], axis=-1)
        bgr = ColorComposite.stretch(bgr, blim=self.blim, glim=self.glim, rlim=self.rlim)
        return bgr


class NDVI(object):
    def __init__(self, red='B04_20', nir='B8A'):
        self.red = red
        self.nir = nir

    def __call__(self, ds):
        ds = (ds[self.nir] - ds[self.red]) / (ds[self.nir] + ds[self.red] + 0.0000001)
        return ds


def np2ipw(arr):
    img = PILImage.fromarray((arr * 255).astype(np.uint8))
    img = img.resize(size=(arr.shape[0] * 4, arr.shape[1] * 4), resample=PILImage.NEAREST)

    with io.BytesIO() as fileobj:
        img.save(fileobj, 'PNG')
        img_b = fileobj.getvalue()
    img_w = ipw.Image(value=img_b)
    return img_w
        

class Interface(object):
    def __init__(self, cube, features,
                 compositor=ColorComposite(),
                 vi_calculator=NDVI(),
                 window_size=500):
        """
        Args:
            point (shapely.geometry.Point): A point in the crs of the cube
        """
        self.cube = cube
        self.features = features
        self.compositor = compositor
        self.vi_calculator = vi_calculator
        self.window_size = window_size
        self.current_idx = 0

        # Widgets
        self.next_button = ipw.Button(description='Next')
        self.previous_button = ipw.Button(description='Previous')
        self.label_input = ipw.Text(description='Label')
        self.output_area = ipw.Output()

        # Bind events
        self.next_button.on_click(self.next_sample)
        self.previous_button.on_click(self.previous_sample)
        #self.label_input.observe(self.label_changed, names='value')

        # Initialize display
        self.update_display()

    def next_sample(self, b):
        # Go to next sample
        if self.current_idx < len(self.features) - 1:
            self.current_idx += 1
            self.update_display()

    def previous_sample(self, b):
        # Go to previous sample
        if self.current_idx > 0:
            self.current_idx -= 1
            self.update_display()

    def update_display(self):
        with self.output_area:
            clear_output(wait=True)
            print(f"Sample {self.current_idx + 1}/{len(self.features)}")
            chips = self.get_chips(self.current_idx)
            ts = self.get_ts(self.current_idx)
            box_layout = ipw.Layout(
                display='flex',
                flex_flow='row wrap',
                align_items='stretch',
                width='100%',
                height='800px',  # Set a fixed height (modify as needed)
                overflow='auto'  # Add scrollability
            )
            box = ipw.Box(children=chips, layout=box_layout)
            interface = ipw.VBox([ts, box])
            display(interface)

    def display_interface(self):
        # Display the full interface
        display(ipw.HBox([self.previous_button, self.next_button]))
        display(self.output_area)

    def get_chips(self, idx):
        geom = self.features[idx]['geometry']
        point = shape(geom)
        bbox = point.buffer(self.window_size).bounds
        cube_sub = self.cube.rio.clip_box(*bbox)
        imgs = []
        for date in cube_sub.time.values:
            slice = cube_sub.sel(time=date)
            bgr = self.compositor(slice)
            imgs.append(np2ipw(bgr))
        return imgs
        

    def get_ts(self, idx):
        cube_sub = self.cube.sel(x=self.features[idx]['geometry']['coordinates'][0],
                                 y=self.features[idx]['geometry']['coordinates'][1],
                                 method='nearest')
        ds = self.vi_calculator(cube_sub)
        # Create scales
        x_sc = bqplot.LinearScale()
        y_sc = bqplot.LinearScale()
        # Create axes
        x_ax = bqplot.Axis(label='Time', scale=x_sc, tick_format='%m-%Y', tick_rotate=45)
        y_ax = bqplot.Axis(label='Vegetation Index', scale=y_sc, orientation='vertical', side='left')
        # Create line mark
        vi_values = bqplot.Scatter(x=ds.time.values, y=ds.values, scales={'x': x_sc, 'y': y_sc})  
        # Create and display the figure
        fig = bqplot.Figure(marks=[vi_values], axes=[x_ax, y_ax], title='Sample temporal profile')
        fig.layout.width = '1400px'
        fig.layout.height = '400px'
        return fig

interface = Interface(cube=cube, features=fc)
interface.display_interface()

        

    

Output()

In [22]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import sqlite3

class DataLabelingInterface:
    def __init__(self, data_samples):
        self.data_samples = data_samples
        self.current_index = 0

        # SQLite Database Setup
        self.conn = sqlite3.connect(':memory:')
        self.cursor = self.conn.cursor()
        self.cursor.execute("CREATE TABLE labels (id INTEGER PRIMARY KEY, label TEXT)")
        self.cursor.executemany("INSERT INTO labels (label) VALUES (?)", [(label,) for label in [''] * len(data_samples)])


        # Widgets
        self.next_button = widgets.Button(description='Next')
        self.previous_button = widgets.Button(description='Previous')
        self.label_input = widgets.Text(description='Label')
        self.output_area = widgets.Output()

        # Bind events
        self.next_button.on_click(self.next_sample)
        self.previous_button.on_click(self.previous_sample)
        self.label_input.observe(self.label_changed, names='value')

        # Initialize display
        self.update_display()

    def update_display(self):
        # Update the display area with the current sample
        with self.output_area:
            clear_output(wait=True)
            print(f"Sample {self.current_index + 1}/{len(self.data_samples)}: {self.data_samples[self.current_index]}")
            label = self.get_label(self.current_index)
            self.label_input.value = label
            display(self.label_input)

    def display_interface(self):
        # Display the full interface
        display(widgets.HBox([self.previous_button, self.next_button]))
        display(self.output_area)

    def next_sample(self, b):
        # Go to next sample
        if self.current_index < len(self.data_samples) - 1:
            self.current_index += 1
            self.update_display()

    def previous_sample(self, b):
        # Go to previous sample
        if self.current_index > 0:
            self.current_index -= 1
            self.update_display()

    def label_changed(self, change):
        # Update label in database for current sample
        self.update_label(self.current_index, change['new'])

    def update_label(self, index, label):
        # Update the label in the database
        self.cursor.execute("UPDATE labels SET label = ? WHERE id = ?", (label, index + 1))
        self.conn.commit()

    def get_label(self, index):
        # Retrieve the label from the database
        self.cursor.execute("SELECT label FROM labels WHERE id = ?", (index + 1,))
        result = self.cursor.fetchone()
        return result[0] if result else ''

# Sample data
data_samples = [
    "The quick brown fox jumps over the lazy dog",
    "A stitch in time saves nine",
    "An apple a day keeps the doctor away",
    "Early to bed and early to rise, makes a man healthy, wealthy, and wise"
]

# Create and display the labeling interface
labeling_interface = DataLabelingInterface(data_samples)
labeling_interface.display_interface()



Output()

In [3]:
import bqplot
import xarray as xr
import numpy as np
import pandas as pd

# Create a time range
time = pd.date_range('2020-01-01', periods=12, freq='M')

# Create a 3D array: 12 time points, 5x5 grid
nir = np.random.rand(12, 5, 5)  # Simulated NIR data
red = np.random.rand(12, 5, 5)  # Simulated Red data

# Create an xarray Dataset
ds = xr.Dataset(
    {
        "nir": (("time", "y", "x"), nir),
        "red": (("time", "y", "x"), red)
    },
    coords={
        "time": time,
        "y": np.arange(5),
        "x": np.arange(5)
    }
)

# Select a single pixel and compute NDVI
pixel_nir = ds.nir.isel(x=2, y=2)
pixel_red = ds.red.isel(x=2, y=2)
ndvi = (pixel_nir - pixel_red) / (pixel_nir + pixel_red)

# Using bqplot's object model API

# Create scales
x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

# Create axes
x_ax = bqplot.Axis(label='Time', scale=x_sc, tick_format='%m-%Y', tick_rotate=45)
y_ax = bqplot.Axis(label='NDVI', scale=y_sc, orientation='vertical', side='left')

# Create line mark
ndvi_line = bqplot.Scatter(x=ndvi.time.values, y=ndvi.values, scales={'x': x_sc, 'y': y_sc})

# Create and display the figure
fig = bqplot.Figure(marks=[ndvi_line], axes=[x_ax, y_ax], title='NDVI Time Series')
fig.layout.width = '600px'
fig.layout.height = '400px'
fig


Figure(axes=[Axis(label='Time', scale=LinearScale(), tick_format='%m-%Y', tick_rotate=45), Axis(label='NDVI', …